In [1]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np

from histomicstk.preprocessing.color_normalization import deconvolution_based_normalization
from histomicstk.saliency.tissue_detection import (
    get_slide_thumbnail, get_tissue_mask)
from skimage.transform import resize

from utils.helpers import compare_two_images, cut_out_image

In [2]:
image_paths = sorted(["data/images/"+path for path in os.listdir("data/images") if not path.startswith(".")])

In [3]:
images = list(map(tifffile.imread, image_paths))

In [4]:
os.listdir("data/reference/")

['E2+P4+DHT_4_M14_3L_0004.tif', 'E2+DHT_1_M13_3L_0001.tif']

In [39]:
# reference_image_1 = tifffile.imread("data/reference/E2+DHT_1_M13_3L_0001.tif")
# reference_image_2 = tifffile.imread("data/reference/E2+P4+DHT_4_M14_3L_0004.tif")
# compare_two_images(reference_image_1, reference_image_2)

In [29]:
def create_mask(img):
    mask_out, _ = get_tissue_mask(
    img, deconvolve_first=True,
    n_thresholding_steps=1, sigma=1.5, min_size=30)

    mask_out_fixed = resize(
    mask_out == 0, output_shape=img.shape[:2],
    order=0, preserve_range=True) == 1

    return ~mask_out_fixed

In [20]:
reference_mask_1 = create_mask(reference_image_1)
reference_mask_2 = create_mask(reference_image_2)

In [38]:
# cut_out_reference_1 = cut_out_image(reference_image_1, reference_mask_1)
# cut_out_reference_2 = cut_out_image(reference_image_2, reference_mask_2)
# compare_two_images(cut_out_reference_1, cut_out_reference_2)

In [22]:
def color_normalization_macenko(target_img, reference_img, mask=None, stains=['hematoxylin', 'eosin'], stain_unmixing_method='macenko_pca'):
    stain_unmixing_routine_params = {
        'stains': stains,
        'stain_unmixing_method': stain_unmixing_method
    }

    normalized_target_img = deconvolution_based_normalization(
            target_img, im_target=reference_img,
            mask_out = mask,
            stain_unmixing_routine_params=stain_unmixing_routine_params,)

    return normalized_target_img

In [32]:
masks = [create_mask(img) for img in images]

In [24]:
# compare_two_images(images[5], masks[5])

### Test Reference Image 1 without cutout

In [79]:
normalized_images_1 = [color_normalization_macenko(image, reference_image_1, masks[i]) for i, image in enumerate(images)]

In [92]:
# for img, norm_img in zip(images, normalized_images_1):
#     plt.figure(figsize=(12, 6))

#     plt.subplot(1, 2, 1)
#     plt.axis(False)
#     plt.title("Original image")
#     plt.imshow(img)

#     plt.subplot(1, 2, 2)
#     plt.axis(False)
#     plt.title("Normalized image")
#     plt.imshow(norm_img)

#     plt.show()

### Test reference image 2 without cutout

In [35]:
normalized_images_2 = [color_normalization_macenko(image, reference_image_2, masks[i]) for i, image in enumerate(images)]

In [37]:
# for img, norm_img in zip(images, normalized_images_2):
#     plt.figure(figsize=(12, 6))

#     plt.subplot(1, 2, 1)
#     plt.axis(False)
#     plt.title("Original image")
#     plt.imshow(img)

#     plt.subplot(1, 2, 2)
#     plt.axis(False)
#     plt.title("Normalized image")
#     plt.imshow(norm_img)

#     plt.show()

### Test reference image 1 with cut out

In [83]:
normalized_images_1_cut = [color_normalization_macenko(image, cut_out_reference_1, masks[i]) for i, image in enumerate(images)]

In [94]:
# for img, norm_img in zip(images, normalized_images_1_cut):
#     plt.figure(figsize=(12, 6))

#     plt.subplot(1, 2, 1)
#     plt.axis(False)
#     plt.title("Original image")
#     plt.imshow(img)

#     plt.subplot(1, 2, 2)
#     plt.axis(False)
#     plt.title("Normalized image")
#     plt.imshow(norm_img)

#     plt.show()

### Test reference 2 with cutout

In [85]:
normalized_images_2_cut = [color_normalization_macenko(image, cut_out_reference_2, masks[i]) for i, image in enumerate(images)]

In [95]:
# for img, norm_img in zip(images, normalized_images_2_cut):
#     plt.figure(figsize=(12, 6))

#     plt.subplot(1, 2, 1)
#     plt.axis(False)
#     plt.title("Original image")
#     plt.imshow(img)

#     plt.subplot(1, 2, 2)
#     plt.axis(False)
#     plt.title("Normalized image")
#     plt.imshow(norm_img)

#     plt.show()

Reference image 2 without cutout seems to provide the best results!